# Index  
1. [Download Data](#Download-Data)
2. [Load Data](#Laod-Data)
3. [Explore Data](#Explore-Data)
4. [Extract hourly entry/exit rate](#Extract-hourly-entry/exit-rate)
5. [Clean Data](#Clean-Data)
6. [Visualize Data](#Visualize-Data)
7. [Conclusion](#Conclusion)
8. [Appendix](#Appendix)

# Download Data

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import regex

def get_data(start, end, file_name="MTA_data", desc='Data Download', auto_save=True):
    """
    input-
    start: Tha start date as int, formatted as followed yymmdd (e.g.: 2020, March 27 is 200327).
    end: Tha end date as int, formatted as followed yymmdd (e.g.: 2020, March 27 is 200327).
    file_name: The desired name for the csv file.
    desc: Progress bar message.
    auto_save: If True, reset index and save file. Otherwise don't save and return df as is.
    
    Output-
    DataFrame with all weeks stacked on eachother.
    
    Notes:
    - Start value must be smaller than end value.
    - If download can't be started wait a couple hours then try again, it could be due to API restrictions.
    - The dates have to be preasent in the website Example: 200327 (2020, March 27) is valid, 
    while 200322 (2020, March 22) is not valid because it does not exist in the website.
    
    EXAMPLE:
    get all data from 2020, january 1 until 2020, march 31.
    start= 191228 (2019, december 28) because the first entry for january in 2020 is 4th of january.
    end= 200404 (2020, april 04) because the last entry for march in 2020 is 28th of march.
    
    call the function with the dates:
    get_data(191228, 200404)
    
    MTA dataset website: http://web.mta.info/developers/turnstile.html
    
    Developed By: Hazim Bukhari
    """
    
    #get page html
    req = requests.get("http://web.mta.info/developers/turnstile.html")
    soup = BeautifulSoup(req.text, "html.parser")
    
    #extract all files links from the html, then filter it
    all_links=[link.get('href') for link in soup.find_all('a')[37:]]
    use_links=[ i for i in all_links if int( regex.search('[0-9]{6}', i).captures()[0] ) >= start and int( regex.search('[0-9]{6}', i).captures()[0] ) <= end ]
      
    #download and build the dataframe
    sub_link='http://web.mta.info/developers/'
    df=pd.read_csv(f"{sub_link}{use_links[0]}")
    for i in tqdm(use_links[1:], desc=desc):
        df= pd.concat([df, pd.read_csv(f"{sub_link}{i}")])
       
    #fix index issue and save file
    if auto_save:    
        df.reset_index(inplace=True)
        df.drop('index', axis=1, inplace=True)
        df.to_csv(f"{file_name}.csv")
    
    return df

df= get_data(160903, 160917,'MTA_2020jan_2020dec', auto_save=False)



# Load Data

In [99]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://data.db', echo=False)
df= pd.read_sql("SELECT * FROM table", engine)

In [383]:
df= pd.read_csv('MTA_2020jan_2020dec.csv')
df.rename(columns={df.columns[-1]:df.columns[-1].strip()}, inplace=True) #fix a typo in the EXIT column name

# Explore Data

In [387]:
df.describe()[['ENTRIES', 'EXITS']]

,ENTRIES,EXITS
count,1.123738e+07,1.123738e+07
mean,4.264399e+07,3.494368e+07
std,2.181730e+08,1.981686e+08
min,0.000000e+00,0.000000e+00
25%,2.846100e+05,1.233030e+05
50%,1.794894e+06,1.045896e+06
75%,6.451383e+06,4.321239e+06
max,2.128903e+09,2.123516e+09


### comments  
Both ENTRIES and EXITS are cumulatives which means no negative values should be preasent, which is observed. 


## Extract hourly entry/exit rate  
Since cumulative sums don't tell us how many people have entered throught the turnstyle, we have to take the difference between two cumulatives sums to get this information.  

Note: The compination of C/A, UNIT, and SCP point to a unique turnstyle.

In [393]:
entry_exit= df.groupby(['C/A', 'UNIT', 'SCP', 'STATION','DATE', 'TIME'])[['ENTRIES', 'EXITS']].max().diff()
entry_exit

ENTRIES  EXITS
C/A   UNIT SCP      STATION       DATE       TIME                    
A002  R051 02-00-00 59 ST         01/01/2020 03:00:00      NaN    NaN
                                             07:00:00      7.0   11.0
                                             11:00:00     31.0   39.0
                                             15:00:00    118.0   40.0
                                             19:00:00    182.0   57.0
...                                                        ...    ...
TRAM2 R469 00-05-01 RIT-ROOSEVELT 12/31/2020 04:00:00      0.0    0.0
                                             08:00:00      0.0    0.0
                                             12:00:00      0.0    0.0
                                             16:00:00      0.0    0.0
                                             20:00:00      0.0    0.0

[11237280 rows x 2 columns]

In [394]:
# check the new dataframe
entry_exit.describe()

,ENTRIES,EXITS
count,1.123728e+07,1.123728e+07
mean,-6.516242e-01,-2.209779e-01
std,1.116464e+07,1.163245e+07
min,-2.115572e+09,-2.109493e+09
25%,1.000000e+00,2.000000e+00
50%,1.600000e+01,1.600000e+01
75%,6.400000e+01,5.800000e+01
max,2.120207e+09,2.123280e+09


### comments  
Why is there negative values? this does not make sence!!  
Through missing with data I found 3 reasons for negative values, which are the following:  

1- The value of the first date of the second turnstyle is substracted from the value from the last date of the first turn style. refere to [Reason 1 example](#Negative-Values-Reason1).  
  
2- The counter starts substracting instead of adding. Why it does that is unknown it could be an overflow, bug, or anything else. refere to [Reason 2 example](#Negative-Values-Reason2).

3- This data has a very unique property which that it resets on random times with random values which is lower than the last value before the reset. note: I had it shown but I deleted by mistake so i am going to refrence an article i read on the matter [article](https://medium.com/analytics-vidhya/mta-turnstile-data-simple-exploratory-data-analysis-a46324d3ca96).  
>The count of a turnstile can be reset at anytime and to any number. One would expect the reset behavior would mean >to reset to zero. However, after observing a few examples, there are turnstiles that seemed to be “reset” to a >random number lower than the previous count.  

### Solution  
Since there are 3 reasons we gonna have 3 solutions which addresses each reason respectively:  

1- Take the diff for each unique turnstyle separately.  

2- The turnstyle is still counting just backward so take the absloute value to make it positive.  

3- Finally put a limit to the difference between two values if it goes above that limit assume it is a reset or new unknown error and set it to 0. Keep in mind that a turnstyle CAN'T admit more than 14400 person per 4 hours. This [article](https://www.avant-gardeturnstiles.com/turnstile-throughput-considerations/) talks about the throughput of a turn style, and the best of them has a throughput of one person per 1-2 seconds.

# Clean Data  
Before doing any Visualizing the miss with the negative numbers has to be sorted, the below code deals with that miss using the solution proposed in the previous section.

In [9]:
#Uncomment for clean data
#import pandas as pd
#df= pd.read_csv('dataV2.csv')

C:\Users\Me\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [338]:
#create datetime column for date based minupliation
print('stage1:\nCreate datetime column\n')
df['datetime']= df[['DATE', 'TIME']].apply(lambda x : x[0]+" "+x[1], axis=1)
df['datetime']= pd.to_datetime(df['datetime'], format='%m/%d/%Y %H:%M:%S')

#create turn style column for turnstyle based operations (diff), and filter out RECOVER AUD entries
print('stage2:\nCreate turnstyle column and delete rows with RECOVER AUD\n')
df['turnstyle']= df[['C/A', 'UNIT', 'SCP', 'STATION']].apply(lambda x: f"{x[0]}-{x[1]}-{x[2]}-{x[3]}", axis=1)
df = df[df['DESC']!= 'RECOVR AUD']

#Sort dataframe on turnstyle and datetime to easely impelement the first solution
print('stage3:\nSort dataframe on turnstyle and datetime\n')
df.sort_values(['turnstyle', 'datetime'], inplace=True)
df.reset_index(inplace=True ,drop=True)


df['entry_diff']= np.zeros(len(df),)
jan['exit_diff']= np.zeros(len(df),)
from tqdm import tqdm

for i in tqdm(jan['turnstyle'].unique(), desc='This is going to take a while, go grap coffee and some snacks'):
    #ENTRIES
    t= df[df['turnstyle']==i][['ENTRIES']].diff()
    df['entry_diff'].loc[t.index]=t.values.reshape(t.values.shape[0])
    #EXISTS
    t= df[df['turnstyle']==i][['EXITS']].diff()
    df['exit_diff'].loc[t.index]=t.values.reshape(t.values.shape[0])

#ENTRIES
df['entry_diff']= df['entry_diff'].abs()
ind= df[df['entry_diff']>=14000]['entry_diff'].index
val= np.zeros(ind.shape[0],).astype(int)
df['entry_diff'].loc[ind]=val

#EXISTS
df['exit_diff']= df['exit_diff'].abs()
ind= df[df['exit_diff']>=14000]['exit_diff'].index
val= np.zeros(ind.shape[0],).astype(int)
df['exit_diff'].loc[ind]=val

df.fillna(0, inplace=True)

#Extract features from DATE for filtring
df['year']= df['DATE'].apply(lambda x : int(x.split('/')[-1]))
df['month']= df['DATE'].apply(lambda x : int(x.split('/')[0]))
df['day']= df['DATE'].apply(lambda x : int(x.split('/')[-2]))

stage1:
stage2:
stage3:


100%|██████████| 5078/5078 [55:19<00:00,  1.53it/s] 
C:\Users\Me\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## Comments  
Now all the values are positive and are capped at 14000 which is the maximum therotical throuput for 4 hours for current turnstyles. Now let's go to the visualizing part.

# Visualize Data  
Now the fun part starts, finally we can find which stations are most crowded for 2020

In [82]:
x[::-1]

Index(['TIMES SQ-42 ST', 'FULTON ST', '86 ST', '125 ST', '14 ST-UNION SQ',
       '42 ST-PORT AUTH', '23 ST', '34 ST-HERALD SQ', 'GRD CNTRL-42 ST',
       '34 ST-PENN STA'],
      dtype='object', name='STATION')

In [85]:
import plotly.graph_objects as go


data = df[df['year']==2020].groupby('STATION')['entry_diff'].sum().sort_values(ascending=False)[:10]
x=data.index
y=data.values

#uncomment to use pre-processed data
#import pandas as pd
#data= pd.read_csv('top10entry.csv')
#x=data['STATION']
#y=data['entry_diff']

fig = go.Figure(data=[go.Bar(
            x=y[::-1], y=x[::-1],
            text=y[::-1],
            orientation='h'
        )])

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', 
                  xaxis_title='Total riders', yaxis_title='Stations', title='Top 10 crowded stations 2020 entry wise')

fig.show()

In [86]:
data = df[df['year']==2020].groupby('STATION')['exit_diff'].sum().sort_values(ascending=False)[:10]
x=data.index
y=data.values

#uncomment to use pre-processed data
#data= pd.read_csv('top10exit.csv')
#x=data['STATION']
#y=data['exit_diff']

fig = go.Figure(data=[go.Bar(
            x=y[::-1], y=x[::-1],
            text=y[::-1],
            orientation='h'
        )])

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', 
                  xaxis_title='Total riders', yaxis_title='Stations', title='Top 10 crowded stations 2020 exit wise')

fig.show()

In [31]:
#Are the top 10 stations the same for entry and exit?
d1= df[df['year']==2020].groupby('STATION')['exit_diff'].sum().sort_values(ascending=False)[:10]
d2= df[df['year']==2020].groupby('STATION')['entry_diff'].sum().sort_values(ascending=False)[:10]

[True if i in d2.index else False for i in d1.index]

[True, True, True, True, True, True, True, True, True, True]

## Comments  
To make sure that thses results make sence let's compare them to the [MTA 2020 most busy stations](https://neweast.mta.info/agency/new-york-city-transit/subway-bus-ridership-2020)  

| Rank | Station               | Ridership  |
|------|-----------------------|------------|
| 1    | Times Sq-42 St        | 20,341,240 |
| 2    | Grand Central-42 St   | 13,162,826 |
| 3    | 34 St-Herald Sq       | 12,826841  |
| 4    | 14 St-Union Sq        | 10,830,712 |
| 5    | Fulton St             | 8,855,302  |
| 6    | 34 St-Penn Station    | 8,103,809  |
| 7    | 34 St-Penn Station    | 8,010,472  |
| 8    | 59 St-Columbus Circle | 7,618,925  |
| 9    | 74-Broadway           | 7,523,538  |
| 10   | Flushing-Main St      | 6,944,923  |  

We might not have the same order of stations due to the following reasons:  
1- The MTA has a cleaner version of the data.  
2- The MtA calculate the ridership in a different manner than us.  

We might not have identical results to the MTA but at least we have numbers that are close to the real numbers.  

Note: To strengthen the validity of point #2 notice that the MTA counted "34 St-Penn Station" twice.

In [98]:
data = pd.read_csv('landuse.txt')
data2=data['landuse'].value_counts()
x=data2.index
y=(data2.values/len(data))*100


fig = go.Figure(data=[go.Bar(
            x=y[::-1], y=x[::-1],
            text=y[::-1],
            orientation='h'
        )])

fig.update_traces(texttemplate='%{text:.2s}%', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', 
                  xaxis_title='Percentage', yaxis_title='Land use', title='Percentage of land use for top 10 stations')

fig.show()

## Comments  
1. Most of stations are located on/near land that is used for commercial building or office building which makes sence since people need to work all year round.  

2. Also it explains why the top 10 for entry and exit are identical, since those people who exited the station to go to work need to enter it again to go back home.  

3. Finally as to why stations in housing land use don't get much traffic, simply all these employees work at the same building but live in different parts in the city.

[source for new york land use](https://zola.planning.nyc.gov/#13.33/40.74823/-73.98644)

# Conclusion  
Based on the MTA Data, the most crowded stations in Riyadh's are most likly going to be stations near/in work places, right now the obvious stations are:
1. King Abdullah Financial Center Station
2. King Abdulaziz City for Science and Technology

other stations are hard to pin down without in depth analysis.

Something intresting about the MTA top 10 stations is that almost none is near Universities, why is that?  
it could be because of students living on campus and that students schedule are not necessarily fixed like employees but does the same translate to Universities in KSA? only time can tell. 

# Appendix

## Negative Values Reason1

In [191]:
jan[['turnstyle', 'datetime','ENTRIES', 'entry_diff','entry_diff_fixed']][180:200]

,turnstyle,datetime,ENTRIES,entry_diff,entry_diff_fixed
180,A002-R051-02-00-00-59 ST,2020-01-31 03:00:00,7362707,35.0,35.0
181,A002-R051-02-00-00-59 ST,2020-01-31 07:00:00,7362720,13.0,13.0
182,A002-R051-02-00-00-59 ST,2020-01-31 11:00:00,7362832,112.0,112.0
183,A002-R051-02-00-00-59 ST,2020-01-31 15:00:00,7363054,222.0,222.0
184,A002-R051-02-00-00-59 ST,2020-01-31 19:00:00,7363791,737.0,737.0
185,A002-R051-02-00-00-59 ST,2020-01-31 23:00:00,7364037,246.0,246.0
186,A002-R051-02-00-01-59 ST,2020-01-01 03:00:00,6511722,-852315.0,NaN
187,A002-R051-02-00-01-59 ST,2020-01-01 07:00:00,6511725,3.0,3.0
188,A002-R051-02-00-01-59 ST,2020-01-01 11:00:00,6511745,20.0,20.0
189,A002-R051-02-00-01-59 ST,2020-01-01 15:00:00,6511864,119.0,119.0


## Negative Values Reason2

In [218]:
jan[898030:898047]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,year,month,day,turnstyle,datetime,entry_diff,entry_diff_fixed
898030,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/04/2020,08:00:00,REGULAR,1506661704,1313904795,2020,01,04,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-04 08:00:00,-4.300000e+01,-4.300000e+01
898031,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/04/2020,12:00:00,REGULAR,1506661581,1313904750,2020,01,04,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-04 12:00:00,-1.230000e+02,-1.230000e+02
898032,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/04/2020,16:00:00,REGULAR,1506661450,1313904652,2020,01,04,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-04 16:00:00,-1.310000e+02,-1.310000e+02
898033,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/04/2020,20:00:00,REGULAR,1506661328,1313904543,2020,01,04,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-04 20:00:00,-1.220000e+02,-1.220000e+02
898034,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/05/2020,00:00:00,REGULAR,1506661301,1313904445,2020,01,05,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-05 00:00:00,-2.700000e+01,-2.700000e+01
898035,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/05/2020,04:00:00,REGULAR,1506661296,1313904425,2020,01,05,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-05 04:00:00,-5.000000e+00,-5.000000e+00
898036,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/05/2020,08:00:00,REGULAR,1506661267,1313904407,2020,01,05,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-05 08:00:00,-2.900000e+01,-2.900000e+01
898037,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/05/2020,12:00:00,REGULAR,1506661162,1313904369,2020,01,05,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-05 12:00:00,-1.050000e+02,-1.050000e+02
898038,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/05/2020,16:00:00,REGULAR,1506661040,1313904285,2020,01,05,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-05 16:00:00,-1.220000e+02,-1.220000e+02
898039,R646,R110,01-00-01,FLATBUSH AV-B.C,25,IRT,01/05/2020,20:00:00,REGULAR,1506660963,1313904186,2020,01,05,R646-R110-01-00-01-FLATBUSH AV-B.C,2020-01-05 20:00:00,-7.700000e+01,-7.700000e+01
